# Tabular data example

In [1]:
from velour import Dataset, Model, Datum, Annotation, GroundTruth, Prediction, Label
from velour.enums import TaskType
from velour import schemas

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [2]:
from velour.client import Client, ClientException

client = Client("http://localhost:8000")

Successfully connected to host at http://localhost:8000/


In [3]:
dset = load_breast_cancer()
X, y, target_names = dset["data"], dset["target"], dset["target_names"]
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipe = make_pipeline(StandardScaler(), LogisticRegression())

In [4]:
X_train.shape, y_train[:4], target_names

((426, 30), array([0, 1, 1, 1]), array(['malignant', 'benign'], dtype='<U9'))

In [5]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [6]:
y_train_probs = pipe.predict_proba(X_train)
y_test_probs = pipe.predict_proba(X_test)

In [7]:
y_train_probs[:4]

array([[0.99887853, 0.00112147],
       [0.04048628, 0.95951372],
       [0.00100522, 0.99899478],
       [0.00509649, 0.99490351]])

## Velour Dataset ingestion

We now ingest the groundtruth labels into velour. For each sample, velour expects a list of `Label` objects. Each `Label` has a key and value. Allowing key/value labels and having a single row be annotated by multiple labels supports multi-label classification.

In this example there's just a single label per element and we'll set the class key to "class". The `add_groundtruth` method returns the ids of the newly created groundtruth.

Create a Velour `Dataset` for the train and test sets respectively.

In [8]:
# create train dataset
velour_train_dataset = Dataset(client, "breast-cancer-train", reset=True)

# create test dataset
velour_test_dataset = Dataset(client, "breast-cancer-test", reset=True)

Create groundtruths using Velour's `GroundTruth` object.

In [9]:
# format training groundtruths
training_groundtruths = [
    GroundTruth(
        datum=Datum(
            uid=f"train{i}",
        ),
        annotations=[
            Annotation(
                task_type=TaskType.CLASSIFICATION,
                labels=[Label(key="class", value=target_names[t])]
            )
        ]
    )
    for i, t in enumerate(y_train)
]

# format testing groundtruths
testing_groundtruths = [
    GroundTruth(
        datum=Datum(
            uid=f"test{i}",
        ),
        annotations=[
            Annotation(
                task_type=TaskType.CLASSIFICATION,
                labels=[Label(key="class", value=target_names[t])]
            )
        ]
    )
    for i, t in enumerate(y_test)
]

Use `Dataset.add_groundtruth` for data ingestion to the backend. 

In [10]:
# add the training groundtruths
for gt in training_groundtruths:
    velour_train_dataset.add_groundtruth(gt)

# add the testing groundtruths
for gt in testing_groundtruths:
    velour_test_dataset.add_groundtruth(gt)

Finalize datasets, necessary for evaluation

In [11]:
velour_train_dataset.finalize()
velour_test_dataset.finalize()

<Response [200]>

## Model inference ingestion

Now we create a velour model and post the predictions on the two datasets. Each prediction should be a list of `ScoredLabel`, which consist of a label and a confidence score. The confidence scores over all of the classes in a key must sum to (approximately) 1.

Create Velour `Model`

In [12]:
velour_model = Model(client, "breast-cancer-linear-model", reset=True)

Format predictions

In [13]:
training_predictions = [
    Prediction(
        datum=Datum(
            dataset=velour_train_dataset.name,
            uid=f"train{i}",
        ),
        annotations=[
            Annotation(
                task_type=TaskType.CLASSIFICATION,
                labels=[
                    Label(
                        key="class", 
                        value=target_names[j],
                        score=p,
                    )                        
                    for j, p in enumerate(prob)
                ]
            )
        ]
    )
    for i, prob in enumerate(y_train_probs)
]

testing_predictions = [
    Prediction(
        datum=Datum(
            dataset=velour_test_dataset.name,
            uid=f"test{i}",
        ),
        annotations=[
            Annotation(
                task_type=TaskType.CLASSIFICATION,
                labels=[
                    Label(
                        key="class",
                        value=target_names[j],
                        score=p,
                    )                        
                    for j, p in enumerate(prob)
                ]
            )
        ]
    )
    for i, prob in enumerate(y_test_probs)
]

In [14]:
# add the train predictions
for pd in training_predictions:
    velour_model.add_prediction(pd)

# add the test predictions
for pd in testing_predictions:
    velour_model.add_prediction(pd)

finalize models, necessary for evaluation

In [15]:
velour_model.finalize_inferences(velour_train_dataset)
velour_model.finalize_inferences(velour_test_dataset)

evaluate

In [16]:
train_eval_job = velour_model.evaluate_classification(velour_train_dataset)
train_eval_job.wait_for_completion()
results = train_eval_job.results()

In [17]:
results.dataframe

value
dataset                                           breast-cancer-train
type      parameters             label                               
Accuracy  {"label_key": "class"} n/a                         0.983568
F1        "n/a"                  class: benign               0.986817
                                 class: malignant            0.978193
Precision "n/a"                  class: benign               0.981273
                                 class: malignant            0.987421
ROCAUC    {"label_key": "class"} n/a                         0.997381
Recall    "n/a"                  class: benign               0.992424
                                 class: malignant            0.969136

In [18]:
results.confusion_matrices

[{'label_key': 'class',
  'entries': [{'prediction': 'benign', 'groundtruth': 'benign', 'count': 262},
   {'prediction': 'benign', 'groundtruth': 'malignant', 'count': 5},
   {'prediction': 'malignant', 'groundtruth': 'benign', 'count': 2},
   {'prediction': 'malignant', 'groundtruth': 'malignant', 'count': 157}]}]

In [19]:
test_eval_job = velour_model.evaluate_classification(velour_test_dataset)
test_eval_job.wait_for_completion()
results = train_eval_job.results()

In [20]:
results.dataframe

value
dataset                                           breast-cancer-train
type      parameters             label                               
Accuracy  {"label_key": "class"} n/a                         0.983568
F1        "n/a"                  class: benign               0.986817
                                 class: malignant            0.978193
Precision "n/a"                  class: benign               0.981273
                                 class: malignant            0.987421
ROCAUC    {"label_key": "class"} n/a                         0.997381
Recall    "n/a"                  class: benign               0.992424
                                 class: malignant            0.969136

In [21]:
results.confusion_matrices

[{'label_key': 'class',
  'entries': [{'prediction': 'benign', 'groundtruth': 'benign', 'count': 262},
   {'prediction': 'benign', 'groundtruth': 'malignant', 'count': 5},
   {'prediction': 'malignant', 'groundtruth': 'benign', 'count': 2},
   {'prediction': 'malignant', 'groundtruth': 'malignant', 'count': 157}]}]

## Sanity check scikit-learn classification report

In [22]:
from sklearn.metrics import classification_report

In [23]:
y_train_preds = pipe.predict(X_train)

In [24]:
print(classification_report(y_train, y_train_preds, digits=6, target_names=target_names))

              precision    recall  f1-score   support

   malignant   0.987421  0.969136  0.978193       162
      benign   0.981273  0.992424  0.986817       264

    accuracy                       0.983568       426
   macro avg   0.984347  0.980780  0.982505       426
weighted avg   0.983611  0.983568  0.983538       426

